In [3]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import time

In [4]:
def find_center_of_hand(hand_landmarks):
    x_vals = [landmark.x for landmark in hand_landmarks.landmark]
    y_vals = [landmark.y for landmark in hand_landmarks.landmark]
    z_vals = [landmark.z for landmark in hand_landmarks.landmark]

    x_center = sum(x_vals) / len(x_vals)
    y_center = sum(y_vals) / len(y_vals)
    z_center = sum(z_vals) / len(z_vals)

    return x_center, y_center, z_center

In [5]:
def calculate_angle(L_coordinates, R_coordinates):
    x1, y1, z1 = L_coordinates
    x2, y2, z2 = R_coordinates

    x_diff = x2 - x1
    y_diff = y2 - y1
    z_diff = z2 - z1

    cos_theta = x_diff / np.sqrt(x_diff**2 + y_diff**2 + z_diff**2)

    theta_degrees = np.arccos(cos_theta) * 180 / np.pi

    theta_degrees = -theta_degrees if y_diff > 0 else theta_degrees

    return round(theta_degrees, 2)

In [6]:
def draw_landmarks(detection_results, image):

    width = image.shape[1]
    height = image.shape[0]

    for hand_landmarks in detection_results.multi_hand_landmarks:
        x_center, y_center, _ = find_center_of_hand(hand_landmarks)
        cv.circle(image, (int(x_center * width), int(y_center * height)), 5, (255, 0, 0), -1)
        for landmark in hand_landmarks.landmark:
            x = min(int(landmark.x * width), width - 1)
            y = min(int(landmark.y * height), height - 1)
            cv.circle(image, (x, y), 5, (0, 255, 0), -1)

    return image

In [7]:
def flatten_scale_relative(results):

    flattened_scaled_results = []
    for hand_landmarks in results.multi_hand_landmarks:
        x_wrist = hand_landmarks.landmark[0].x
        y_wrist = hand_landmarks.landmark[0].y

        for landmark in hand_landmarks.landmark:
            flattened_scaled_results.append(landmark.x - x_wrist)
            flattened_scaled_results.append(landmark.y - y_wrist)
            flattened_scaled_results.append(landmark.z)
    
    return flattened_scaled_results

In [8]:
def write_to_csv(results, csv_path, category):
    csv_row = [category]
    csv_row.extend(flatten_scale_relative(results))

    # write to csv
    with open(csv_path, 'a') as f:
        f.write(','.join([str(x) for x in csv_row]) + '\n')


In [11]:
# hands = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7,
#                                  min_tracking_confidence=0.5)

# img = cv.imread('Data\istockphoto-1267273871-612x612.jpg')
# img = cv.flip(img, 1)


# cv.imshow('image', img)

# results = hands.process(cv.cvtColor(img, cv.COLOR_BGR2RGB))

# write_to_csv(results, 'Data\hands.csv', 0)

# cv.waitKey(0)
# cv.destroyAllWindows()

In [11]:
def log_data(csv_path, category, range):
    cap = cv.VideoCapture(0)
    hands = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7,
                                    min_tracking_confidence=0.5)

    NUM_SAMPLES = 1000

    time.sleep(7)
    print('STARTING')
    sample_count = 0
    while True:
        success, img = cap.read()
        img = cv.flip(img, 1)

        if not success:
            print("Could not read frame")
            break

        results = hands.process(cv.cvtColor(img, cv.COLOR_BGR2RGB))

        if results.multi_hand_landmarks and sample_count < NUM_SAMPLES:
            
            if len(results.multi_hand_landmarks) == 2:
                # write_to_csv(results, csv_path, category)

                #draw a line between the centers of the two hands
                x_center_L, y_center_L, z_center_L = find_center_of_hand(results.multi_hand_landmarks[0])
                x_center_R, y_center_R, z_center_R = find_center_of_hand(results.multi_hand_landmarks[1])

                cv.line(img, (int(x_center_L * img.shape[1]), int(y_center_L * img.shape[0])),
                        (int(x_center_R * img.shape[1]), int(y_center_R * img.shape[0])), (0, 0, 255), 3)
                
                #calculate angle between the two hands
                angle = calculate_angle((x_center_L, y_center_L, z_center_L), (x_center_R, y_center_R, z_center_R))

                # write angle on top left corner
                cv.putText(img, str(angle), (10, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv.LINE_AA)

                if category == 0:
                    if range[0] < angle < range[1]:
                        sample_count += 1
                        write_to_csv(results, csv_path, category)
                else:
                    sample_count += 1
                    write_to_csv(results, csv_path, category)
                
                            
            if len(results.multi_hand_landmarks) < 2:
                print('NOT WRITING')
            
            img = draw_landmarks(results, img)
        
        cv.imshow("image", img)

        if sample_count == 250 or sample_count == 750:
            print(sample_count)

        if cv.waitKey(1) == 27 or sample_count == NUM_SAMPLES:
            break

    cap.release()
    cv.destroyAllWindows()

In [12]:
# collecting data for steering position
log_data('Data\steering_Angles\15_angle.csv', 0, (-5, 5))

STARTING
250
750
